# 4章 ニューラルネットワークの学習
本章のテーマは、ニューラルネットワークの学習。  
ここで言う「学習」とは、訓練データから最適な重みパラメータの値を自動で獲得することを指す。  
学習を行うためにために本章では*損失関数*という指標を導入する。  
**この損失関数を基準として、その値が最も小さくなる重みパラメータを探し出すことが学習の目的**。

## 4.1 データからの学習

ニューラルネットワークの特徴は、データから学習できること。  
つまり、重みパラメータの値を自動で決定できるということ。  
これができないとニューラルネットワークの設計時に数千、数万のパラメータを手作業で設定しなければならず、現実的ではなくなってしまう。  
  
### データについて

機械学習はデータが中心であり、人間の介入を極力避けている。  
データから何かしらのパターンを認識し、正解を導き出す。  
そのためには入力値から*特徴量*を抽出して、その特徴量のパターンを機械学習の技術で学習する方法が考えられる。  
**特徴量とは、入力値から本質的な値を的確に抽出できるように設計された変換器を指す。**  
例えば画像の特徴量は通常ベクトルとして記述される。  
SIFTやSURF、HOGなどの特徴量を使って画像データをベクトルに変換し、その変換されたベクトルに対して機械学習で使われる識別器（SVMやKNNなど）で学習させることができる。  
学習自体は自動で行われるが、特徴量の設計やその選択は人間に依存しているため、問題ごとに最適な特徴量を考える必要がある。（例えば手書きの5を識別するための特徴量を犬の識別に使っても役に立たない）  
  
それに対してニューラルネットワークでは、入力データをそのまま学習する。  
つまり、特徴量を人為的に設計した機械学習と違い、ニューラルネットワークでは画像に含まれる重要な特徴量までも自動で学習してくれる。  

### 訓練データとテストデータ
  
機械学習では訓練データとテストデータに分けて学習や実験を行うのが一般的。  
→学習時に使ったデータ以外で汎用性を検証するため。（過学習を避けるため）  

## 4.2 損失関数

ニューラルネットワークの学習では、一つの指標を手がかりに最適なパラメータを模索する。  
この指標のことを*損失関数*と呼ぶ。  
損失関数とは「性能の悪さ」を表しており、これを最小化することで学習をすすめる。
損失関数は任意の関数を用いることができるが、一般的には二乗和誤差や交差エントロピー誤差などが用いられる。  
  
### 4.2.1 二乗和誤差
  
$E=\frac{1}{2}\sum_{k}(y_{k}-t_{k})^{2}$  
  
$y_{k}$はニューラルネットワークの出力、$t_{k}$は教師データを表し、$k$はデータの次元数を表す。  

In [1]:
import numpy as np
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [2]:
# the answer is 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# example 1: 2 is most likely(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [3]:
# example 2: 7 is most likely(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
mean_squared_error(np.array(y), np.array(t))

0.5975

1つめの例（正解）が2つ目の例（不正解）よりも二乗和誤差の数値が低くなっていることがわかる。  
誤差が小さいということなので、つまり出力結果が教師データにより適合していることを表している。

### 4.2.2 交差エントロピー誤差
  
$E=-\sum_{k}t_{k}\log y_k$  
  
$\log$は底が$e$の自然対数を表す。  
$y_{k}$はニューラルネットワークの出力、$t_{k}$はone-hot表現の正解ラベルを表す。  
そのため上記の式は正解ラベルが1に対応する出力の自然対数を計算するだけになる。  
（グラフは本書P90を参照）  
$x$が1のとき$y$は0になり、$x$が0に近づくと$y$の値はどんどん小さくなる。  
**つまり、上記の式は正解ラベルに対する出力が大きくなるほど0に近づき、1のとき0になる。  
また、正解ラベルに対する出力が小さければ、値が大きくなるという性質を持つ。**

In [8]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

 ここでは、`np.log`の計算時に微小な値である`delta`を足している。  
 これは`np.log(0)`となった場合に`np．log（0）`がマイナス無限大になり、計算が止まってしまうのを避けるための処置。

In [9]:
# the answer is 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# example 1: 2 is most likely(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [10]:
# example 2: 7 is most likely(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

### 4.2.3 ミニバッチ学習
機械学習の場合、訓練データに対する損失関数を求め、その値を最小化するパラメータを探すので、**損失関数は全ての訓練データを対象として求める必要がある**。（例えば訓練データが100個あれば、その100個の損失関数の和を指標とする）  
例えば交差エントロピー誤差の場合次のような式になる。  
  
$E=-\frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log y_{nk}$  
  
データが$N$個あるとして、$t_{nk}$は$n$個目のデータの$k$次元目の値を意味する。  
（$y$はニューラルネットワークの出力、$t$は教師データ）  
ここでは**$N$で割ることで正規化し、1個あたりの平均の損失関数を求めている。**  
そうすれば、訓練データの数に関係なく、いつでも統一した指標が得られる。
  
数千万といったビッグデータの場合、全てのデータを対象にして損失関数の和を求めるのは時間がかかる。  
そこで、**データの中から一部を取り出し、それをデータ全体の近似として利用する。**（バッチ処理）  
ニューラルネットワークでも訓練データからある枚数だけを選び出し（ミニバッチと言う）、ミニバッチごとに学習を行う。

### 4.2.4 【バッチ対応版】交差エントロピー誤差の実装
  
ここではデータが一つの場合と、データがバッチサイズごとにまとめられて入力される場合の両方のケースに対応するように実装する。  

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

もしも教師データがone-hot表現で与えられずにラベルとして与えられた場合、次のように実装する。

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

`np.arange(batch_size)`は0から`batch_size`-1までの配列を生成する。  
例えば`batch_size`が5だとしたら、`np.arange(batch_size)`は[0,1,2,3,4]。  
そして`t`にはラベルが[2, 7, 0, 9, 4]のように格納されているので、  
`y[np.arange(batch_size), t]`は、各データの正解ラベルに対応するニューラルネットワークの出力を抽出する。  
（この例だと[ y[0, 2], y[1, 7], y[2, 0], y[3, 9], y[4, 4] ]）  

#### なぜ認識精度ではなく損失関数を指標に用いるのか？
  
Q:損失関数の最小化ではなく、認識精度の最大化を指標にすればいいのでは？  
  
A:最大化、最小化は微分が0になることで判断し、微分した値の符号によって変化の方向性を決めるが、認識精度を微分してもほとんどの場合0になるから。  
  
認識精度はパラメータの微小な変化にはほとんど反応を示さず、もし反応があったとしても、その値は不連続かつ唐突に変化するから。同じ理由でステップ関数をニューラルネットワークの活性化関数にすると学習がうまく行えない。一方シグモイド関数のように連続的に変化する曲線の場合は、曲線の傾きも連続的に変化するため、どこで微分しても0にはならない。
  